In [1]:
import pandas as pd
import re
import os
from time import time

In [2]:
years = sorted(os.listdir('abstracts'))

files = []  # attribu en POO ?
nb_files = 0
i = 0

for year in years:
    files.append(os.listdir(f'abstracts/{year}'))
    nb_files += len(files[i])
    i += 1

print(nb_files)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'abstracts'

On va prendre le nom du premier fichier et le nom du dernier fichier pour les itérations.

In [ ]:
time0 = time()
list_authors = []
list_papers = []
for year, file_by_year in zip(years, files):
    for file in sorted(file_by_year):
        with open(f"abstracts/{year}/{file}","r") as f:
            
            j=0 # for pass when there is two lines with Authors => 9201039.abs
            
            for line in f:
                if j < 1:
                    if (line[:9] == "Authors: "):
                        j += 1
                        list_authors.append(re.split(' and |, |& ', line[9:-1]))

                    if line[:8] == "Author: ": 
                        j+= 1
                        list_authors.append(re.split(' and |, | & ', line[8:-1]))

                    if line[:7]=="Paper: ":
                        list_papers.append(line[14:21])
                        
execution_time = time() - time0
print(execution_time)

In [ ]:
for paper, author in zip(list_papers, list_authors):
    print(f"ligne {paper} : ", author)
    
# ligne 9201047 :  ['18 pages'] à remplacer par Dieter Lust and Carlos Munoz
# ligne 9206083 :  ['J.A. Casas', "F. G\\'omez", 'C. Mu\\~noz'] tild

Pourquoi a-t-on bien traité le fichier ?

- Le nombre d'auteurs et de papiers correspond à la somme des fichiers dans les dossier abstracts/{year}


In [ ]:
df = pd.DataFrame({ 'id_paper':list_papers, 'Authors':list_authors})

On obtient donc un DataFrame avec les numéro des papiers en index et une colonne contenant les auteurs dans une liste.
On se pose alors une question : Que se passe t'il si les auteurs sont appelé de différentes manières dans les articles?

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.Authors[37]

In [ ]:
df.Authors[38]

In [ ]:
df.Authors[39]

Maintenant on va compiler tout nos résultat dans une seule fonction qui va retourner ce qui peut nous intéresser.

In [ ]:
def pre_processing_abstact():
    '''
    Pre-processing du fichiers abstracts. 
    Retourne une liste de listes des liens.
    '''

Est ce que on prend en compte que les data peuvent changer ou on considère que les data ne change pas? Réponse : Oui, il y aura de nouvelles data mais elle seront formatée pareils.

In [ ]:
time0 = time()
list_authors = []
list_papers = []
for year, file_by_year in zip(years, files):
    for file in sorted(file_by_year):
        with open(f"abstracts/{year}/{file}","r") as f:
            
            j=0 # for pass when there is two lines with Authors => 9201039.abs
            
            for line in f:
                if j < 1:
                    if line[:9] == "Authors: ":
                        j += 1
                        list_authors.append(re.split(' and |, |& ', line[9:-1]))

                    if line[:8] == "Author: ": 
                        j+= 1
                        list_authors.append(re.split(' and |, |& ', line[8:-1]))

                    if line[:7]=="Paper: ":
                        list_papers.append(line[14:21])
                        
dico = {'papers' : list_papers, 'authors': list_authors}
execution_time = time() - time0
print(execution_time)

## Construction d'un dico Publication par auteur

In [ ]:
from collections import defaultdict 
dico2 = defaultdict(list)
for paper in dico["papers"][:5]:
    for author in dico["authors"][:5]):
    for author in authors:
        print(authors)
        dico2[auteur].append(paper)
dico2

In [ ]:
def pre_processing_abstracts():
    dict_p = defaultdict(list)
    dict_a = defaultdict(list)

    for year, file_by_year in zip(years, files):
        for file in sorted(file_by_year):
            with open(f"abstracts/{year}/{file}","r") as f:
                nb_line_author = 0 # for pass when there is two lines with Authors => 9201039.abs
                for line in f:
                    if nb_line_author < 1:
                        if line[:7]=="Paper: ":
                            tmp_paper = line[14:21]

                        if line[:9] == "Authors: ":
                            nb_line_author += 1
                            dict_p[tmp_paper] = re.split(' and |, |& ', line[9:-1])

                        if line[:8] == "Author: ": 
                            nb_line_author +=  1
                            dict_p[tmp_paper] = re.split(' and |, |& ', line[8:-1])

    for paper, authors in dict_p.items():
        for author in authors:
            dict_a[author].append(paper)
    print("Abstracts files' pre-processing is done.")
    return dict_p, dict_a

pre_processing_abstracts()[1]

In [ ]:
len(dict_a)

In [ ]:
len(dict_p)

In [ ]:
def quote0(author):
    """
    fonction quote avec une profondeur N = 1
    """
    author_quoted = []
    paper_quoted = dict_a[author]
    for paper in paper_quoted:
        for author in dict_p[paper]:
            if not author in author_quoted:
                author_quoted.append(author)
    return author_quoted

print(quote0("C. Itzykson"))

In [ ]:
def quote(author, N=1):
    """
    Chercher pour chaque auteur qui la influencé à son tour mais si auteur dans profondeur n-1 ne pas ajouter.
    Aussi voir la question de la pondération.
    """
    author_quoted = []
    for profondeur in range(N):
        author_quoted.append(quote0(author))